# Week 7: More plotting and some statistics

Today we continue to load data tables from research project and plot them. We also learn to compute basic statistics on a data table like averages and standard deviations. Here is the table of contents:
1. **Showing data collection sites on a map**  
2. **Tracking wolves**  
   Visualize an animal's motion on a map.  
3. **Pelagic megafauna** (optional)  
4. **Sponge recruitment**  
5. **Algae growth, statistics, box plots**  
6. **Microhabitats**  
   Compute and visualize statistics about the reef microhabitat we discussed last week.

## 1. Showing data collection sites on a map

The mplleaflet module makes it very easy to plot things on a map. To make a scatter plot on a map, just make a regular scatter plot with `plt.scatter` using the longitudes as x coordinates and the latitudes as y coordinates. At the end, instead of `plt.show()`, call `mplleaflet.show()`. That will create a file called `_map.html` and open it in your web browser like a regular webpage.

In [ ]:
# Import the necessary modules
import pandas as pd              # To load spreadsheets.
import matplotlib.pyplot as plt  # To plot.
import mplleaflet                # To plot on maps.

In [ ]:
# Now we load the data. It's in an excel spreadsheet so we use the read_excel function 
# from the pandas module. The names of the columns are read from the first row.
data = pd.read_excel('oil_sample_locations.xlsx')
display(data)

In [ ]:
# Now we plot. First we make a regular scatter plot with pyplot using the longitude column
# as x coordinates and the latitude column as y coordinates, then we call mplleaflet.show().
# This creates the file _map.html which is then opened in your browser like a regular webpage.
# Note that you need internet access to see the map. The _map.html file contains your points, 
# but it doesn't contain the map per say, only a link to an online map. 
plt.scatter(data['LONGITUDE'],data['LATITUDE'],color='r',s=100)
mplleaflet.show()

In [ ]:
# You can change the map's style with the "tiles" option.
# The default style is 'osm' (open street map).
# To get topographic lines use tiles='esri_worldtopo'.
# To get aerial views use tiles='esri_aerial'.
plt.scatter(data['LONGITUDE'],data['LATITUDE'],color='r',s=100)
mplleaflet.show(tiles='esri_aerial')

In [ ]:
# Here is the full list of available styles:
print(list(mplleaflet.maptiles.tiles.keys()))

## 2. Tracking wolves

This is from an ongoing research project at the University of Minnesota Twin Cities. It involves placing GPS collars on wolves living in the Voyageur National Park. Check out the project's website at https://www.voyageurswolfproject.org, there's much more to it. 

The voyageur wolf collaboration generously accepted to share the data with us, but they ask that you **<font color='red'>do not share the data</font>** outside of this class as it could be used to harm the wolves.

The data files are located in the `wolf_data` folder. Each file corresponds to a different wolf. They're all csv files with the column names in the first row.

In [ ]:
# Here is the wolf "V028", whose location was recorded every 20 minutes for 6 months.
# Note the name of the file is now a path. "wolf_data/V028.csv" means the file "V028.csv"
# located in the folder "wolf_data", itself located in the same folder as this notebook.
data = pd.read_csv('wolf_data/V028.csv')
display(data)

In [ ]:
# Let's plot it on a map.
plt.plot(data['Longitude'],data['Latitude'])
mplleaflet.show()

Next we'd like to show every wolf's trajectory on the same map. The first step is to make a list of all the wolves' data files. You can do that by hand or use the function `glob` from module `glob`, which creates a list of files matching a certain pattern:

In [ ]:
import glob
print(glob.glob('wolf_data/*.csv'))
# The pattern 'wolf_data/*.csv' matches any file with extension '.csv' in the folder 'wolf_data'.
# The output of glob.glob is a list of every file that matches the pattern.

<div class="alert alert-block alert-danger">
<b>Task 1:</b>
Write a loop over the wolf data files. For each file, load the data and plot the wolf's trajectory. Once the loop is over, use mplleaflet to render the whole thing on a map.
</div>

## 3. Pelagic Megafauna (optional)

<div class="alert alert-block alert-danger">
<b>Optional Task 1:</b>
Download Dataset 1 from https://www.nature.com/articles/s41598-018-38144-8#additional-information. Load it into pandas. Create a set of mplleaflet maps showing the trajectory of a fish, one map per pair of longitude-latitude columns. For each fish you'll need to discard any row with a missing longitude or latitude (look up "pandas dropna"). Name each map after the species the data is from using `mplleaflet.save_html(fileobj='name_of_the_file.html')` instead of `mplleaflet.show()`.
</div>

## 4. Sponge recruitment

See Andia's slides about sponge recruitment.

No new python/pandas concepts here, just practicing what we've already learned.

<div class="alert alert-block alert-danger">
<b>Task 2:</b>
Load the data in `sponge_recruits.csv`. Use it to make a pie chart showing the proportion of each species' recruits. Use labels and a title to make it self-explanatory.
</div>

## 5. Algae growth, statistics, and box plots

See Andia's slides about algae growth and statistics.

The data for this section is in the excel spreadsheet `algae_growth.xlsl`. Algae are grown in the lab under various conditions. The growing conditions are recorded in the "conditions" column. The height to which the algae grow is recorded in centimeters in the "growth" column. Each row corresponds to a different experiment. Each type of growing conditions was tested multiple times in no particular order.

In [ ]:
# Load the data.
data = pd.read_excel('algae_growth.xlsx')
display(data)

### 5.1. Column statistics

Pandas' method `describe` provides an overview of a data table's main statistical properties.

In [ ]:
display(data.describe())
# The output is a new data table. 
# It has 8 rows, one for each statistical property being computed. 
# It has the same columns as the original data table, at least the ones that contain numbers.
# In this case the 'conditions' column got ignored because it contains strings for which
# there's no clear way to compute a mean, standard deviation, etc.

There are also functions to compute individual statistical properties. It's simplest to use them on a single column, then the answer is a simple number.

In [ ]:
# Print the mean of the "growth" column.
print('Mean:')
print(data['growth'].mean())

# Print the standard deviation of the "growth" column.
print('\nStandard deviation:')
print(data['growth'].std())

We can also compute statistical properties that are not computed by `describe`. The most important one is the standard error, which is used to assess the uncertainty of the mean:

In [ ]:
print('Standard error:')
print(data['growth'].sem())

### 5.2. Grouping by growing conditions

The order of the spreadsheet's rows often reflects the order in which the experiments were performed and the results recorded. What we really want to know, though, is whether and how the growth rate depends on the growing conditions. To do that, we need to group the rows by growing conditions.

Pandas' method `groupby` does exactly that. All we need to tell it is the name of the column containing the information we want to group the rows by. In this case it's the `conditions` column.

In [ ]:
grouped_data = data.groupby('conditions')

# The output of "groupby" is kind of like a data table but not quite. 
# Unfortunately, it cannot be displayed like a table:
display(grouped_data)

In [ ]:
# On the other hand we can request one specific group and display that
# like a table:
display(grouped_data.get_group('Caged')) # Filtered data table with only the 'Caged' rows.

More importantly, we can use any of pandas' statistical functions (count, mean, std, sem, etc) on the output of `groupby` to compute the corresponding statistical property for each group. The result is a data table with one row for each group and one column for each numerical column of the original table (in this case just the "growth" column).

In [ ]:
means = data.groupby('conditions').mean()
display(means)

# Unlike the data tables we've worked with before, in this one the rows
# have names shown in bold on the left.
# Note that "conditions" is not a true column of the table anymore,
# rather it's a list of the rows' names.

In [ ]:
# As with any pandas data table we can extract a column by name:
display(means['growth'])

In [ ]:
# We can get the list of row names using the table's "index" property:
display(means.index)

In [ ]:
# To extract a specific value from a column we can call it either by
# its row name or by its row index.
print('Mean growth in the control group: ')
print(means['growth']['Control']) # Specify row by name.
print(means['growth'][2]) # Specify row by index.

### 5.3. Plotting group statistics

We would now like to make a bar plot of the mean growth height for each growing conditions with error bars showing the uncertainty of the mean as obtained from the standard error.

In [ ]:
grouped_data = data.groupby('conditions')

# We can compute the means like this:
means = grouped_data.mean()
display(means)
# However that is a data table, and we can't plot a data table
# against another, only a column against another. Hence we must
# specify we want the 'growth' column:
means = grouped_data.mean()['growth']
print(means)

# Same thing for the standard error, which we'll use to scale the error bars:
serrs = grouped_data.sem()['growth']

# For the horizontal axis of the plot we need the list of conditions,
# which we can get as the row names of either 'means' or 'serrs':
conditions = means.index

# The "plt.figure" needs to go before any plotting command. I'm using it to control 
# the size of the graph. The two numbers between brackets after "figsize=" are the 
# width and height of the plot. Play around with them until you find a size you like.
plt.figure(figsize=[10,6])
# Now we're ready to plot. The 'yerr' option is used the specify the
# size of the error bars (by default there are no error bars at all).
plt.bar(conditions,means,yerr=serrs)
plt.ylabel('Height (cm)')
plt.title('Algae height grown under various conditions')
plt.xticks(rotation=45)
plt.show()

### 5.4. Making a box plot

A lot goes into making a box plot, including grouping the data, computing its quartiles, its outliers, etc. However it's such a common need that pandas has a function that takes care of everything for you. All you need to specify is which column you want on the vertical axis (with the `column` option) and which column you want on the horizontal axis (with the `by` option). Under the hood `boxplot` uses the latter to group the data then computes the quartiles of the former for each group.

In [ ]:
# By default boxplot draws a grid in the background. I didn't like it so I turned it off
# with "grid=False". You can delete the option of change False to True if you want.
# I also increased the size of the graph with the figsize option. Note that the "plt.figure"
# trick doesn't work to set the size of pandas boxplots, instead you have to use the "figsize"
# option inside the "boxplot" command. As before the two numbers are the width and height of 
# the graph.
data.boxplot(column='growth',by='conditions',grid=False,figsize=[10,6])
# For some weird reason this plot has two titles that overlap each other. 
# One is controlled by "plt.suptitle", the other by "plt.title". Here I get rid of the
# former by setting it to an empty string then use the latter for the actual title.
plt.suptitle('')
plt.title('Algae height grown under various conditions')
plt.xlabel('Growing conditions')
plt.ylabel('Height (cm)')
plt.xticks(rotation=60) # Tilting x axis labels by 60 degrees.
# plt.ylim(0,15)
plt.show()

Sometimes you need to zoom in on a specific range of values. To do that you can use `plt.xlim` and `plt.ylim`. We discussed examples of that last week; here is another one:

In [ ]:
data.boxplot(column='growth',by='conditions',grid=False)
plt.suptitle('')
plt.title('Algae height grown under various conditions')
plt.xlabel('Growing conditions')
plt.ylabel('Height (cm)')
plt.ylim(4.5,12.5)
plt.show()

## 6. Microhabitats

Let us now apply this new knowledge of statistics and box plots to the microhabitat data from last week.

<div class="alert alert-block alert-danger">
<b>Task 3:</b>
Load the data in `microhabitats.xlsx`. Make a list of means and a list of standard errors across all microhabitats and all years. Make a bar plot and a box plot, and in one paragraph explain what both plots suggest about the percent cover of microhabitats. Which microhabitat has more percent cover? Which one less?

For this next (optional) task you need to know how to group the data then loop over all the groups.

In [ ]:
# To get the list of all group names, retrieve the column used for grouping
# then eliminate duplicates with "unique":
covers = data['type of cover'].unique()
print(covers)

# Note: Don't use "type" as a variable name, it's a python reserved keyword.

# At this point if you wanted to compute something for each group you could loop
# over the groups. For basic stats there is a simpler way, but this can be useful
# for more advanced use cases. Here I'm just printing a separate table for each 
# growing conditions to illustrate the method.
for cover in covers:
#     display(data[data['conditions']==condition])
    display(data.groupby('type of cover').get_group(cover))

You also need to know about `plt.errorbar`. It works pretty much like `plt.plot` except there's a `yerr` option you can use to set the vertical error bars like we did in `plt.bar`. Here is a made up example:

In [ ]:
plt.errorbar([1,2,3],[4,3,5],yerr=[0.5,0.3,0.1])
plt.show()

<div class="alert alert-block alert-danger">
<b>Optional Task 2:</b>
Load the data in `microhabitats.xlsx`. Make a line plot with error bars (using `plt.errorbar`) of the percent coverage against the year. Make a separate curve for each type of cover, but put them all on the same graph. What does the graph tell us about the evolution of reef microhabitats between 2003 and 2018?